In [1]:
import json
import matplotlib.pyplot as plt
from ipywidgets import Label, HBox 
from IPython.display import display
import matplotlib.image as mpimg

In [3]:
# Load the rink image with the correct path
rink_image = mpimg.imread('../figures/nhl_rink.png')

FileNotFoundError: [Errno 2] No such file or directory: '../figures/nhl_rink.png'

In [154]:
# Function to load event data from JSON files
def load_event_data(season, game_type):
    filename = f'ift6758/data/{season}/{game_type}.json'
    with open(filename) as f:
        return json.load(f)

In [156]:
# Create widgets for selecting season and match
season_dropdown = Dropdown(options=[str(year) for year in range(2016, 2025)], description='Season:')
match_dropdown = Dropdown(options=[], description='Match:')
output_plot = Output()

In [158]:
def update_match_options(change):
    season = change['new']
    print(f"Loading data for season: {season}")

    # Load regular season data
    season_data = load_event_data(season, 'season')  
    print(f"Regular Season data for {season}:")
    print(json.dumps(season_data, indent=2))  
    # Create match options, ensuring id exists
    match_options = [game['id'] for game in season_data if 'id' in game]
    
    # Load playoff data
    playoffs_data = load_event_data(season, 'playoffs')
    print(f"Playoffs data for {season}:")
    print(json.dumps(playoffs_data, indent=2))  

    # Append playoff match options
    playoff_match_options = [game['id'] for game in playoffs_data if 'id' in game]
    
    # Combine regular and playoff match options
    match_options.extend(playoff_match_options)

    print(f"Match options for season {season}: {match_options}")  

    # Update the match dropdown options
    match_dropdown.options = match_options

season_dropdown.observe(update_match_options, names='value')

In [160]:
def plot_events_on_rink(change):
    match_id = change['new']
    season = season_dropdown.value  # Get the currently selected season
    print(f"Selected match ID: {match_id} for season: {season}")

    # Load the event data for the selected match
    season_data = load_event_data(season, 'season')  
    # Load playoff data
    playoffs_data = load_event_data(season, 'playoffs')

    with output_plot:
        output_plot.clear_output(wait=True)  # Clear previous output
        fig, ax = plt.subplots()
        ax.imshow(rink_image)  # Display the rink image

        # Find the game in regular season
        game_found = False # track if the game is found
        for game in season_data:
            if game.get('id') == match_id:  # avoid KeyError
                events = game.get('plays', [])
                game_found = True
                for event in events:
                    if 'details' in event and 'xCoord' in event['details'] and 'yCoord' in event['details']:
                        x = event['details']['xCoord']
                        y = event['details']['yCoord']
                        ax.scatter(x, y, color='red')  # Plot the event
                    else:
                        print(f"Event with no coordinates: {event}")  # Print metadata 

        # If not found in regular season, check playoffs
        if not game_found:
            for game in playoffs_data:
                if game.get('id') == match_id:
                    events = game.get('plays', [])
                    game_found = True
                    for event in events:
                        if 'details' in event and 'xCoord' in event['details'] and 'yCoord' in event['details']:
                            x = event['details']['xCoord']
                            y = event['details']['yCoord']
                            ax.scatter(x, y, color='red')  # Plot the event
                        else:
                            print(f"Event with no coordinates: {event}")  # Print metadata 

        if not game_found:
            print(f"No game found with ID: {match_id}")  # Print if no game matches the ID

        plt.show()  # Show the plot

match_dropdown.observe(plot_events_on_rink, names='value')


In [170]:
# Create labels 
season_label = Label(value="Choose a Season:")
match_label = Label(value="Choose a Match:")

# Display the widgets and output area 
layout = VBox([
    HBox([season_label, season_dropdown]),
    HBox([match_label, match_dropdown]),
    output_plot
])
display(layout)